In [2]:
import json
import os
from typing import Any, Dict, NamedTuple
import urllib

import requests

In [78]:
_HEADERS = {
    'User-Agent': 'Aggregit',
}

## My repo

In [197]:
r = requests.get('https://api.bitbucket.org/2.0/repositories/pygame')

In [198]:
r.json().keys()

dict_keys(['pagelen', 'values', 'page', 'size'])

## Sample record

In [199]:
r.json()['values'][0]

{'created_on': '2011-08-03T11:32:56.437822+00:00',
 'description': 'Moved to github. https://github.com/pygame/pygame',
 'fork_policy': 'allow_forks',
 'full_name': 'pygame/pygame',
 'has_issues': True,
 'has_wiki': True,
 'is_private': False,
 'language': 'python',
 'links': {'avatar': {'href': 'https://bitbucket.org/pygame/pygame/avatar/32/'},
  'branches': {'href': 'https://api.bitbucket.org/2.0/repositories/pygame/pygame/refs/branches'},
  'clone': [{'href': 'https://bitbucket.org/pygame/pygame', 'name': 'https'},
   {'href': 'ssh://hg@bitbucket.org/pygame/pygame', 'name': 'ssh'}],
  'commits': {'href': 'https://api.bitbucket.org/2.0/repositories/pygame/pygame/commits'},
  'downloads': {'href': 'https://api.bitbucket.org/2.0/repositories/pygame/pygame/downloads'},
  'forks': {'href': 'https://api.bitbucket.org/2.0/repositories/pygame/pygame/forks'},
  'hooks': {'href': 'https://api.bitbucket.org/2.0/repositories/pygame/pygame/hooks'},
  'html': {'href': 'https://bitbucket.org/pygam

## Helper functions

In [ ]:
def get_all_items(endpoint):
    all_items = []

    url = endpoint # first page
    while True:
        r = requests.get(url)

        all_items.extend(r.json()['values'])

        if 'next' in r.json():
            url = r.json()['next']
        else:
            return all_items

## Get all data

In [109]:
def get_all_items(endpoint):
    all_items = []

    url = endpoint # first page
    while True:
        r = requests.get(url)

        all_items.extend(r.json()['values'])

        if 'next' in r.json():
            url = r.json()['next']
        else:
            return all_items

In [172]:
all_repos = get_all_items('https://api.bitbucket.org/2.0/repositories/alysivji')

In [173]:
originals = [repo for repo in all_repos if 'parent' not in repo]
forks = [repo for repo in all_repos if 'parent' in repo]

assert len(originals) + len(forks) == len(all_repos)

In [174]:
print(f'Total Num of Original Repos: {len(originals)}')
print(f'Total Num of Forked Repos: {len(forks)}')

Total Num of Original Repos: 11
Total Num of Forked Repos: 2


In [175]:
watcher_endpoints = [repo['links']['watchers']['href'] for repo in all_repos]

In [176]:
total_watchers = 0

for endpoint in watcher_endpoints:
    r = requests.get(endpoint, headers=_HEADERS)
    total_watchers += r.json()['size'] # pull from metadata

In [177]:
print(f'Total Num of Watchers: {total_watchers}')

Total Num of Watchers: 12


In [178]:
issues_endpoints = [repo['links']['issues']['href']
                    for repo in all_repos
                    if repo['has_issues'] is True]

In [179]:
all_issues = []

for endpoint in issues_endpoints:
    issues = get_all_items(endpoint)
    all_issues.extend(issues)

In [180]:
open_issues = [issue for issue in all_issues
               if issue['state'] in ['new', 'open']]

In [181]:
print(f'Total Num of Open Issues: {len(open_issues)}')

Total Num of Open Issues: 0


In [185]:
commit_endpoints = [repo['links']['commits']['href']
                    for repo in all_repos
                    if 'parent' not in repo]

In [187]:
all_commits = []

for endpoint in commit_endpoints:
    commits = get_all_items(endpoint)
    all_commits.extend(commits)

total number of commits to their repos (not forks) total size of their accounts
a list/count of langgauges used
a list/count of repo topics

In [190]:
print(f'Total Number of Commits in public (non-forked) repos: {len(all_commits)}')

Total Number of Commits in public (non-forked) repos: 11


In [195]:
repo_sizes = [repo['size'] for repo in all_repos]
total_size = sum(repo_sizes)

In [196]:
print(f'Total Size of Account: {total_size}')

Total Size of Account: 20822093


In [208]:
repo_languages = [repo['language'] for repo in all_repos]

languages = set(repo_languages)
if '' in languages:
    languages.remove('')

print(f'Repo languages: {list(set(languages))}')

Repo languages: ['python']
